## Terrorisme en de Media



### Introductie

Deze data story volgt een bevestigende benadering. We zullen twee datasets analyseren en visualiseren om bestaande hypotheses te toetsen en overtuigende argumenten te presenteren ter ondersteuning van de verschillende perspectieven die we gedurende deze data story zullen behandelen.

Onze focus ligt op de escalatie van terrorisme in zowel westerse als niet-westerse landen en de potentiele invloed van de media. We zullen licht schijnen op het feit dat de media veel van de aanslagen in het westen laat zien en weinig van het niet-westen. Deze veronderstelling wordt ondersteund door eerdere studies (Jetter, 2014), die hebben aangetoond dat de media zich voornamelijk richt op terrorisme in het westen.

Tijdens onze analyse zullen we rigoureus de gegevens onderzoeken en opvallende patronen en trends benadrukken. We maken gebruik van gegevensvisualisaties om helder inzicht te verschaffen in de manifestatie van terrorisme en eventuele verschillen tussen westerse en niet-westerse landen. Daarnaast zullen we de focus leggen op de berichtgeving in de media en onderzoeken hoe deze mogelijk van invloed is op het publieke beeld van terrorisme.

Met deze data story streven we ernaar om een dieper inzicht te bieden in de complexiteit rondom terrorisme en de rol van de media bij het vormen van percepties. We willen aantonen dat er meerdere perspectieven bestaan en dat het essentieel is om een breed scala aan informatiebronnen te raadplegen om een volledig beeld te krijgen. Door middel van een bevestigende benadering willen we bestaande hypothesen toetsen en concrete inzichten verschaffen die kunnen bijdragen aan een meer gefundeerde discussie over terrorisme en mediaberichtgeving.

In deze datastory zullen we twee perspectieven behandelen. Het eerste perspectief is gebaseerd op de constatering dat terrorisme vaker voorkomt buiten het Westen. Deze benadering benadrukt dat terroristische activiteiten zich wereldwijd manifesteren en dat er regionale en geopolitieke factoren zijn die bijdragen aan de verspreiding ervan. Het tweede perspectief werpt een licht op de rol van de media en stelt dat de berichtgeving over terrorisme voornamelijk gericht is op het Westen. Dit perspectief suggereert dat de media een selectieve focus hebben op terroristische incidenten in westerse landen, omdat deze gebeurtenissen doorgaans meer aandacht trekken en een grotere impact hebben op de publieke opinie.

### Dataset en preprocessing

Voor deze datastory zijn twee datasets gebruikt, namelijk de 'Global Terrorism Database' en de 'Population'-dataset. 

De 'Global Terrorism Database' bevat een groot aantal variabelen die niet relevant zijn voor ons datastory. Daarom hebben we deze irrelevante variabelen verwijderd, waardoor we een kleinere dataset hebben verkregen die specifieke informatie bevat over terroristische aanslagen die wereldwijd hebben plaatsgevonden. Deze gefilterde dataset bevat gegevens over verschillende variabelen die betrekking hebben op deze aanslagen, zoals het jaar van de aanslag, het land waar de aanslag heeft plaatsgevonden, het succes van de aanslag, of het een zelfmoordaanslag was en het aantal gewonden bij de aanslag.

De 'Population'-dataset bevat gegevens over de bevolking van verschillende landen. Deze dataset bevat twee belangrijke variabelen: 'county_txt' en 'population'. De variabele 'county_txt' geeft de naam van het land weer, terwijl de variabele 'population' het aantal inwoners van dat land aangeeft.

Voor ons datastory hebben we alleen deze twee variabelen behouden, aangezien ze relevant zijn voor het analyseren van de relatie tussen de bevolking van een land en terroristische aanslagen. Door gebruik te maken van deze dataset kunnen we inzicht krijgen in de populatiedynamiek en hoe deze mogelijk van invloed kan zijn op het voorkomen van terroristische activiteiten.

Verdere data preprocessing was nodig om de westerse en niet-westerse landen te scheiden, zodat we deze twee kunnen vergelijken in onze visualizaties.

Bovendien hebben we twee functies aangemaakt, de ene `cases_count()` retourneert het totale aantal aanslagen door de jaren heen en de andere `cases_sum()` retourneert de som voor het aantal succes, zelfmoord en verlengde gevallen (apart). Deze hebben we nodig voor onze eerste twee visulaizaties.

In [1]:
'''
Nb. for the sake of memory on GithubPages, we have modified the global terrorism database and filtered the columns accoording to our needs. 
This step is discarded in the code and it has been done before starting with the actual preprocessing of the data. 
So now we have only 14 columns instead of 135.
'''

# Import packages
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

# Load global terrorism and population datasets using pandas
df_1 = pd.read_csv('globalterrorismdb_0718dist.csv', encoding = 'latin-1', low_memory = False)
df_2 = pd.read_csv('Population.csv')

# Data preprocessing
df_1['west_or_not'] = df_1['region'].apply(lambda x: '1' if x == 1 or x == 12 or x == 8 else '0')
df_1['Event'] = 1

population = df_2.sort_values(by=['country_txt'])

filtered = df_1.filter(items=['country', 'Event', 'success', 'nkill', 'nwound'])
grouped = filtered.groupby("country").sum()
grouped = grouped.sort_values(by=['country'])

filteredwest = df_1.filter(items=['country', 'country_txt', 'west_or_not'])
groupedwest = filteredwest.drop_duplicates(subset=['country'])
groupedwest = groupedwest.sort_values(by=['country'])

totalset = groupedwest.join(grouped, how="inner", on="country")
totalset = totalset.merge(population, how='inner', on='country_txt')
totalset = totalset.drop(columns=['country', 'World', '#']) 

totalset = totalset.reset_index()
totalset = totalset.drop(columns=["index"])

west = totalset[totalset['west_or_not'] != '0']
nonwest = totalset[totalset['west_or_not'] != '1']


def cases_count(df_1, column_name):

    cases_count = df_1.groupby(column_name)['eventid'].count()
    
    # Create a new DataFrame from the Series object
    cases_count_df = pd.DataFrame({column_name: cases_count.index, 'Aantal gevallen': cases_count.values})

    # Sort the DataFrame by the number of cases ('Aantal gevallen') column in descending order
    cases_count_df = cases_count_df.sort_values('Aantal gevallen', ascending=False)
    
    
    return cases_count_df

def cases_sum(df_1, col_group_by, value):

    cases_sum = df_1.groupby(col_group_by)[value].sum()
    
    # Create a new DataFrame from the Series object
    cases_sum_df = pd.DataFrame({col_group_by: cases_sum.index, value: cases_sum.values})

    # Sort the DataFrame by the number of cases ('Aantal gevallen') column in descending order
    cases_sum_df = cases_sum_df.sort_values(value, ascending=False)
    
    
    return cases_sum_df

### Perspectieven en argumenten

In [2]:

# Total cases
cases_year_count_df = cases_count(df_1,"iyear")

# Extended cases (incident extended more than 24 hours)
extended_cases_sum_df = cases_sum(df_1,'iyear','extended')

# Success cases
success_cases_sum_df = cases_sum(df_1,'iyear','success')

# Suicide cases
suicide_cases_sum_df = cases_sum(df_1,'iyear','suicide')

# Merge columns on "iyear"
cases_by_year_df = pd.merge(cases_year_count_df, extended_cases_sum_df, on = 'iyear', how='left').merge(success_cases_sum_df, on = 'iyear').merge(suicide_cases_sum_df, on = 'iyear')
# Create a line chart using Plotly
cases_by_year_df = cases_by_year_df.sort_values('iyear', ascending=False)

# Create Scatter traces for extended cases and success cases and suicide cases
total_cases = go.Scatter(x=cases_by_year_df['iyear'], y=cases_by_year_df['Aantal gevallen'], mode='lines', name='Totaal gevallen')
success_cases = go.Scatter(x=cases_by_year_df['iyear'], y=cases_by_year_df['success'], mode='lines', name='Succesvolle gevallen')
extended_cases = go.Scatter(x=cases_by_year_df['iyear'], y=cases_by_year_df['extended'], mode='lines', name='Uitgebreide gevallen')
suicide_cases = go.Scatter(x=cases_by_year_df['iyear'], y=cases_by_year_df['suicide'], mode='lines', name='Zelfmoordgevallen')

# Add traces to a single Figure object
fig = go.Figure(data=[total_cases, success_cases, extended_cases, suicide_cases])
fig.update_layout(title='Aantal aanslagen wereldwijd door de jaren heen', xaxis_title='Year', yaxis_title='Cases')
fig.show()


De bovenstaande grafiek toont een opvallend sterke stijgende trend in het aantal aanslagen wereldwijd door de jaren heen. Aan het moment waarop de sterke groei is ingezet zouden we kunnen afleiden dat dit te maken heeft met de media. Rond die tijd is media groter geworden naar aanleding van de opkomst van de mobiele telefoon. De vraag is echter, zijn er daadwerkelijk meer aanslagen gebeurd of is het aantal hetzelfde gebleven en weten we van meer aanslagen door het gebruik van de mobiele telefoon.

Aanslagen die in buiten gebieden gebeurden, zoals dorpen ergens midden in een woestijn, konden wij vroeger erg gemakkelijk niet van horen. Wij hebben namelijk weinig tot geen contact met deze mensen omdat het erg ver weg is. Tegenwoordig kunnen de mensen uit zo'n buitengebied alles posten en gemakkelijk mensen bellen waardoor de informatie snel verspreid wordt.

Een vraag die hieruit afgeleid kan worden; is de hoeveelheid terrorisme gelijk gebleven en weten we het nu meer of is er wat anders aan de hand? Een andere mogelijkheid is namelijk dat er meer terrorisme gebeurd door de betere verspreiding. Als jij jouw mening wilt laten weten kan dit nu makkelijk door middel van een terroristische aanslag. Deze aanslag zal dan verspreid worden en zo ook jouw boodschap erbij. 

In [3]:
cases_count_df = cases_count(df_1, "country_txt")

# create choropleth map
fig = px.choropleth(cases_count_df, 
                    locations='country_txt',
                    locationmode='country names',
                    color='Aantal gevallen',
                    hover_name='country_txt',
                    projection='natural earth')

fig.update_layout(title_text='Aantal aanslagen over de wereld')


fig.show()

In bovenstaand figuur is een wereldkaart afgebeeld met de daarbij het aantal aanslagen wat er in het desbetreffende land is gebeurd. Vanuit het figuur is het duidelijk te zien dat de landen met het hoogste aantal aanslagen zich bevinden in het Midden-Oosten en Azië, en niet in westerse landen. Dit staat haaks op de berichtgeving in de media, die de nadruk legt op aanslagen in het Westen. 
Wel zien wij dat de landen waar veel aanslagen gebeuren ook landen zijn waar oorlog is. Terreur is iets typisch van een oorlog. Een voorbeeld hiervan is Irak. Op de kaart is te zien dat Iraq het land is daar de meeste aanslagen plaats vinden. 

De oorlog in Irak is begonnen door voornamelijk de VS en die kreeg daarin steun van onder andere Nederland. De reden voor het starten van deze oorlog waren mogelijke banden met de terroristische groep Al-Qaida, die verantwoordelijk was voor de aanslag op de twin towers op 11 september 2001. Ook was er desinformatie vanuit de VS dat Irak massavernietigingswapens zou hebben. Propoganda door middel van de media heeft ervoor gezorgt dat deze oorlog escaleerde en er veel doden en gewonden vielen. Ook in Nederland heeft de media gedaan aan propaganda. Duidelijk is dus dat ook de media hier weer een grote rol speelt in het terrorisme.

In [4]:
totalset = totalset.groupby("west_or_not").sum()
totalset['placement'] = ['Non westren country', 'Westren country']
totalset['equated'] = totalset['Event'] / totalset['Population']

labels = totalset['placement']
values1 = totalset['Event']
values2 = totalset['equated']

fig = go.Figure()
colors = ['red', 'green']
fig.add_trace(
    go.Pie(labels=labels, values=values1, hole=.8, textinfo='label+percent'))
fig.add_trace(
    go.Pie(labels=labels, values=values2, hole=.8, textinfo='label+percent', visible=False))

fig = fig.update_layout(
            updatemenus=[
                dict(
                type = 'buttons',
                direction = 'left',
                buttons = list([
                    dict(
                        label = "Origineel",
                        method = "update",
                        args = [{"visibale": [True, False]},
                               {"title": "Aantal aanslagen in westerse en niet westers landen"}]
                    ),
                    dict(
                        label = "Gelijkgesteld",
                        method = "update",
                        args = [{"visible": [False, True]},
                               {"title": "Aantal aanslagen in westerse en niet westerse landen, gelijkgesteld op basis van populatie"}]
                    )
                ])
                )
            ])

fig = fig.update_traces(marker=dict(colors=colors, line=dict(color='#000000', width=1)))
fig = fig.update_layout(title_text='Aantal aanslagen westerse en niet westerse landen', showlegend=False)

fig.show()

De donut chart hierboven geeft het totale aantal aanslagen in de wereld weer en de percentages van de aanslagen in westerse en niet-westerse landen (origineel). Uit deze grafiek blijkt dat bijna 90% van de aanslagen plaatsvond in niet-westerse landen. Op basis hiervan lijkt de media-aandacht voor aanslagen in westerse landen een onlogische keuze, aangezien deze landen slechts iets meer dan 10% van het totale aantal aanslagen vertegenwoordigen. Deze donut chart suggereert dat de media opzettelijk kiest om aanslagen in het westen aan het publiek te tonen.

Echter, wanneer we kijken naar de aangepaste donut chart (waarbij het aantal aanslagen gelijkgesteld is op basis van populatie), zien we dat de verhoudingen van aanslagen slechts 9,4% verschillen. Deze donut chart houdt rekening met het percentage aanslagen in verhouding tot de bevolking. Aangezien het aantal inwoners in niet-westerse landen aanzienlijk groter is dan in westerse landen, kan de keuze van de media om aanslagen in het westen te belichten als "logisch" worden beschouwd, omdat de verhouding tussen aanslagen en bevolking van beide regio's dicht bij elkaar ligt. Het is logisch dat de westerse media eerder aanslagen laat zien uit iets wat dichtbij is en dichtbij mensen staat dan aanslagen die verder weg zijn.



Deze bevindingen werpen een interessant licht op de complexiteit van de media-aandacht voor aanslagen. Hoewel de oorspronkelijke donut chart suggereert dat de media selectief is in het belichten van aanslagen in westerse landen, laat de aangepaste versie zien dat de verhoudingen tussen aanslagen en populatie vergelijkbaar zijn. Het aantal doden en gewonden blijft echter een belangrijke factor bij het bepalen van de media-aandacht. Deze inzichten benadrukken de noodzaak van een kritische benadering en een diepgaand begrip van de verschillende factoren die de media-aandacht voor aanslagen beïnvloeden.

In [5]:
fig = go.Figure()
fig.add_trace(go.Bar(name='Aantal aanslagen per land', x= west['country_txt'], y=west['Event']))
fig.add_trace(go.Bar(name='Aantal doden per land', x= west['country_txt'], y=west['nkill']))
fig.add_trace(go.Bar(name='Aantal succesvolle aanslagen per land', x= west['country_txt'], y=west['success']))

fig = fig.update_layout(
    title="Aantal (succesvolle) aanslagen en gevallen doden in westerse landen",
    autosize=True,
    annotations=[dict(xref='paper',
                    yref='paper',
                    y= -0.45,
                    showarrow=False,
                    text='Westerse landen')])

fig.show()

In bovenstaand figuur worden de westerse landen weergegeven waar de meeste aanslagen hebben plaatsgevonden, samen met het aantal aanslagen per land, het totale aantal doden en het aantal succesvolle aanslagen. Opvallend is dat in de meeste weergegeven landen het aantal aanslagen en het aantal succesvolle aanslagen dicht bij elkaar liggen. Echter, in vier gevallen (VS, Noorwegen, Mexico en Canada) is het aantal doden hoger dan het aantal succesvolle aanslagen. Dit kan mogelijk verklaren waarom de media deze specifieke succesvolle aanslagen meer belicht.

Deze discrepantie tussen het aantal doden en het aantal succesvolle aanslagen in bepaalde landen is een belangrijk aspect om te onderzoeken. Het roept vragen op over de mogelijke factoren die van invloed zijn op de media-aandacht voor aanslagen. Er kunnen verschillende redenen zijn waarom de media zich meer richt op succesvolle aanslagen, zoals de ernstige gevolgen voor het menselijk leven en de maatschappij, de impact op internationale betrekkingen of de perceptie van bedreiging binnen de samenleving.

In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=west["country_txt"].unique(), y=west["nkill"], name="Aantal doodde", mode="lines"))
fig.add_trace(go.Scatter(x=west["country_txt"].unique(), y=west["nwound"], name="Aantal gewonden", mode="lines"))

fig.add_trace(go.Scatter(x=nonwest["country_txt"].unique(), y=nonwest["nkill"], name="Aantal doodde", mode="lines", visible=False))
fig.add_trace(go.Scatter(x=nonwest["country_txt"].unique(), y=nonwest["nwound"], name="Aantal gewonden", mode="lines", visible=False))

fig = fig.update_layout(
            updatemenus=[
                dict(
                type = 'buttons',
                direction = 'left',
                buttons =list([
                    dict(
                        label = "Westerse landen",
                        method="update",
                        args=[{"visible": [True, True, False, False]},
                             {"title": "Aantal doodden en gewonden westerse landen"}]   
                    ),
                     dict(
                        label = "Niet westerse landen",
                        method="update",
                        args=[{"visible": [False, False, True, True]},
                             {"title": "Aantal doodden en gewonden niet westerse landen"}]   
                    )
                ])
                )
            ])


fig.show()

In vergelijking met het vorige figuur wordt het argument met betrekking tot het dodenaantal (en het aantal gewonden) in westerse landen weerlegd door de bovenstaand figuur. Deze visualisatie laat zien dat er in niet-westerse landen een stuk meer doden en gewonden zijn als gevolg van aanslagen in vergelijking met westerse landen. Indien de media hun berichtgeving zou baseren op het aantal doden en gewonden, zouden juist niet-westerse landen meer belicht moeten worden.

Deze observatie roept belangrijke vragen op over de manier waarop de media aanslagen selecteert en belicht. Als het criterium voor berichtgeving inderdaad het aantal slachtoffers is, zou men verwachten dat niet-westerse landen meer aandacht krijgen vanwege de hogere aantallen doden en gewonden. Het feit dat dit niet altijd het geval lijkt te zijn, suggereert wederom dat er mogelijk andere factoren een rol spelen bij de keuze van de media om bepaalde aanslagen te benadrukken.

In [7]:
labels = nonwest['country_txt']
values = nonwest['Event']

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='none',
                            insidetextorientation='radial'
                            )])

fig = fig.update_layout(title = "Procentuele bijdrage per niet-westers land")

fig.show()

De pie chart hierboven toont het aantal aanslagen in niet-westerse landen en geeft aan welk percentage elk land bijdraagt aan het totale aantal aanslagen. Deze visuele weergave benadrukt het belang van niet-westerse landen in termen van het absolute aantal aanslagen.

De top 3 landen met de meeste aanslagen zijn Irak, Pakistan en Afganistan. In alle drie de landen is of was er oorlog. Een oorlog zorgt vanzelfsprekend voor meer terreur en aanslagen. Deze oorlogen zijn al langere tijd gaande. Het is al erg vaak in het nieuws geweest en er zijn weinig nieuwe ontwikkelingen. Dit maakt dat de media ervoor zal kiezen om andere dingen te laten zien. Ook dit zorgt er op zijn tijd weer voor dat er meer in de media te zien is over de aanslagen is het westen.

Echter kan er door de media aandacht voornamelijk te richten op aanslagen in westerse landen een vertekend beeld ontstaan van het wereldwijde terrorisme. Het is essentieel om een evenwichtige en objectieve verslaggeving na te streven die recht doet aan de werkelijke verdeling van aanslagen over de wereld. Dit betekent dat er meer aandacht moet worden besteed aan niet-westerse landen, waar het aantal aanslagen vaak aanzienlijk is.

Het is van belang om de complexiteit van deze kwestie te erkennen en te streven naar een gebalanceerde verslaggeving die recht doet aan de realiteit van terrorisme, ongeacht de geografische locatie. Het bevorderen van een genuanceerde discussie over de rol van de media bij het vormgeven van het publieke bewustzijn is van essentieel belang om een beter begrip te krijgen van de dynamiek rondom aanslagen en de gevolgen ervan op de samenleving als geheel.

### Reflectie


Voor mij (Hussam) was het project een stuk gemakkelijker dan vorig jaar. Ik kon namelijk inspiratie putten uit mijn werk van het voorgaande jaar. Op die manier wist ik hoe ik fouten zoals "Visualization flaws" en "Argument flaws" kon vermijden. Bovendien kon Anna haar werk (vooral de visualisaties) van vorig jaar aanpassen en gebruiken voor dit project. Met deze twee inzichten hadden we een goede start voor het project. Daarna heb ik voornamelijk samengewerkt met Anna om de hele opdracht af te ronden, terwijl de andere twee groepsleden helaas niet effectief hebben bijgedragen aan de opdracht. De communicatie verliep ook niet zoals verwacht. Ik had een groepsapp aangemaakt zodat we allemaal gemakkelijk en snel konden communiceren, maar dat bleek nutteloos te zijn. Ik vind dat we een betere eindversie van het project hadden kunnen inleveren als de samenwerking binnen de hele groep beter was geweest. Bovendien had ik geen mogelijkheid om fysiek aanwezig te zijn voor peer- (of TA-) feedback. Deze gemiste feedback had ons bewust kunnen maken van mogelijke fouten in onze datastory. Desondanks hebben we (vooral Anna) de datastory aangepast op basis van de geschreven feedback die onze TA online heeft gegeven. We hebben de perspectieven duidelijk aangegeven in de inleiding en zijn hier dieper op ingegaan, samen met de bijbehorende argumenten, in de sectie 'Perspectieven en argumenten'. Daarnaast hebben we de volgorde van de visualisaties veranderd, zodat alle visualisaties goed bij elkaar passen en de tekst goed verloopt. Verder heb ik het maximale uit het project gehaald en heb ik er veel van geleerd.

Zelf (Anna) moest ik even schakelen naar de format van dit jaar t.o.v. vorig jaar. Qua communicatie ging het met 1 groepsgenoot erg goed (Hussam), echter ging het met de andere twee niet goed. Ik kreeg weinig gehoor van hen en ze hebben ook erg weinig gedaan.. 
Tim had enkel een beetje tekst geschreven die helaas niet klopte met de opdracht. Vervolgens heeft Mahamed hetzelfde gedaan. Ik heb geprobeerd contact met ze te krijgen helaas ging dit niet. Ik denk dat als zij een actievere werkhouding hadden het beter verlopen was. Nu zaten ze eigenlijk alleen maar Hussam en mij in de weg ipv dat we dit project met zn allen aan het maken waren.

### Werkverdeling

Anna: Code geschreven voor vier visualisaties en tekst geschreven\aangepast.

Hussam: Code geschreven voor twee visualisaties, de tekst continu aangepast, de opmaak van de datastory vastgesteld en alles op Git geregeld.

Tim & Mahamed: Klein beetje tekst geschreven (zie reflectie).

### Referentielijst

1. Jetter, Michael. (2014). Terrorism and the Media. 

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2505359


2. Artikel, 'Precies twintig jaar geleden begon de oorlog tegen Irak, met hulp van propagandaoffensief Nederland'.

https://www.bnnvara.nl/joop/artikelen/precies-twintig-jaar-geleden-begon-de-oorlog-tegen-irak-met-hulp-van-propagandaoffensief-nederland

### Appendix 

Generatieve AI (ChatGPT met GPT-3.5) wordt gebruikt om het project met succesvol af te ronden. De doel van het gebruik van ChatGPT is vooral om de schrijfstijl van de datastory te verbeteren.

In [21]:
import pandas as pd
from IPython.display import display, Markdown

# Maak een lijst met de gegevens voor elke kolom
reasons = ['schrijfstijl verbeteren', 'schrijfstijl verbeteren', 'Een choropleth map maken']
parts = ['Introductie', 'Perspectieven en argumentaties', 'Perspectieven en argumentaties']
prompts = ['"Verbeter de schrijfstijl van de introductie"', '"Verbeter de schrijfstijl van de tekst"', '"how to create a choropleth map using Plotly Express"']

# Maak een DataFrame met de gegevens
data = {'Reden van gebruik': reasons, 'In welke onderdelen?': parts, 'Welke prompts werden gebruikt?': prompts}
df = pd.DataFrame(data)

# Voeg een titel toe aan de DataFrame
df_title = "Overzicht van ChatGPT gebruik"

# Voeg een Markdown-cel toe met de titel en de DataFrame
display(Markdown("Overzicht van ChatGPT gebruik"))
display(df)

Overzicht van ChatGPT gebruik

,Reden van gebruik,In welke onderdelen?,Welke prompts werden gebruikt?
0,schrijfstijl verbeteren,Introductie,"""Verbeter de schrijfstijl van de introductie"""
1,schrijfstijl verbeteren,Perspectieven en argumentaties,"""Verbeter de schrijfstijl van de tekst"""
2,Een choropleth map maken,Perspectieven en argumentaties,"""how to create a choropleth map using Plotly E..."
